In [98]:
'''
import pandas as pd
product_data = pd.read_excel('/home/sam/data/产品模拟 - 调整.xlsx')
product_data.drop(columns=['序号', '编码', '公司名称', '周期', '高折扣', '总中标\n金额', '建信合计', '直投合计', '财通合计',\
                          '诺德合计', '发行日前推报告期'], inplace=True)
product_data.to_csv('product_data.csv')
'''

"\nimport pandas as pd\nproduct_data = pd.read_excel('/home/sam/data/产品模拟 - 调整.xlsx')\nproduct_data.drop(columns=['序号', '编码', '公司名称', '周期', '高折扣', '总中标\n金额', '建信合计', '直投合计', '财通合计',                          '诺德合计', '发行日前推报告期'], inplace=True)\nproduct_data.to_csv('product_data.csv')\n"

In [99]:
'''
import pandas as pd
market_data = pd.read_excel('/home/sam/data/20221206新规后增发实施.xlsx', sheet_name='市场2020-8起')
market_data = market_data.iloc[:701].copy()
market_data.drop(columns=['序号','定增股份上市日', '增发日收盘价', '自发行价涨跌幅(% 后复权)','预计募集资金总额(亿元)', \
                          '实际募资总额(亿元)', '权重', '年度权重', '加权浮盈', '年度权重\n（2020/8-）', \
                          '加权浮盈\n（2020/8-）', '其中：资产募集金额(亿元)','其中：货币募集金额(亿元)', '发行费用(亿元)', \
                          '实际募资净额(亿元)', '发行方式', '发行对象', '认购方式','增发类型', '增发目的', \
                          '增发是否属于简易程序审核','是否控股股东认购', '大股东认购比例(%)', '大股东认购方式', '定价基准日类型',\
                          '定价方式', '初始预案公告日', '定价基准日', '股东大会公告日', '发审委/上市委通过公告日', \
                          '证监会通过公告日','上市公告日', '股权登记日', '主承销商', '财务顾问','证券类型', '公司类型',\
                          '发行代码', '委员会类型'], inplace=True)
market_data.to_csv('market_data.csv')
'''

"\nimport pandas as pd\nmarket_data = pd.read_excel('/home/sam/data/20221206新规后增发实施.xlsx', sheet_name='市场2020-8起')\nmarket_data = market_data.iloc[:701].copy()\nmarket_data.drop(columns=['序号','定增股份上市日', '增发日收盘价', '自发行价涨跌幅(% 后复权)','预计募集资金总额(亿元)',                           '实际募资总额(亿元)', '权重', '年度权重', '加权浮盈', '年度权重\n（2020/8-）',                           '加权浮盈\n（2020/8-）', '其中：资产募集金额(亿元)','其中：货币募集金额(亿元)', '发行费用(亿元)',                           '实际募资净额(亿元)', '发行方式', '发行对象', '认购方式','增发类型', '增发目的',                           '增发是否属于简易程序审核','是否控股股东认购', '大股东认购比例(%)', '大股东认购方式', '定价基准日类型',                          '定价方式', '初始预案公告日', '定价基准日', '股东大会公告日', '发审委/上市委通过公告日',                           '证监会通过公告日','上市公告日', '股权登记日', '主承销商', '财务顾问','证券类型', '公司类型',                          '发行代码', '委员会类型'], inplace=True)\nmarket_data.to_csv('market_data.csv')\n"

# 处理定增因子数据用于选股

In [1]:
'''
import pandas as pd
import numpy as np
quant_data_path = '/home/sam/data/定增因子.xlsx'
raw_data = pd.read_excel(quant_data_path, header=1)

raw_data.dropna(inplace=True)
tmp = pd.to_datetime(raw_data['限售股份解禁日']) - pd.to_datetime(raw_data['发行日期'])
raw_data['持有天数'] = tmp.apply(lambda x:x.days)
raw_data['年化收益'] = raw_data['解禁日绝对收益']*365/raw_data['持有天数']
raw_data.drop(columns=['限售股份解禁日','证监会通过公告日', '最新报告期', '调整报告期', '前次报告期', '基金变化', '年度报告期1',\
                       '上市日期', '解禁日收盘价（定点复权）', '沪深300涨跌幅', '解禁日相对沪深300', '中证500涨跌幅',\
                       '解禁日相对中证500', '行业涨跌幅', '解禁日相对行业' ], inplace=True)

raw_data['发行日期'] = raw_data['发行日期'].astype(np.string_)
raw_data['发行月份'] = raw_data['发行日期'].apply(lambda x:x[:7])

def standardization(quant_col_data):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    quant_factor = pd.DataFrame(scaler.fit_transform(quant_col_data),columns=quant_col_data.columns)
    quant_factor.set_index(quant_col_data.index, inplace=True)
    return quant_factor

continu_col = ['发行价格', '预计募资', '折扣率', '总市值', '增发占增发后自由流通市值', '发行前20日股价涨跌幅', \
               '行业20日涨跌幅', '发行前20日相对行业涨跌幅', '股价', '基金占比', '前次占比', '换手率', '闪发', \
               '净利FY0', '净利FY1', '净利FY2', '未来1年净利润增速', '未来2年净利润增速', 'PS百分位',\
               'PE百分位']
raw_data.replace('-', np.nan, inplace=True)
raw_data.dropna(inplace=True)
raw_data[continu_col] = standardization(raw_data[continu_col])
cond = (raw_data[continu_col] < -3) | (raw_data[continu_col] > 3)
raw_data[continu_col].mask(cond, inplace=True)

from datetime import datetime
start = datetime.strptime('2020-08-10','%Y-%m-%d')
print(raw_data[raw_data['发行日期'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d')) > start].index[0])
train_data = raw_data.loc[:1631]
test_data = raw_data.loc[1631:]
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv',index=False)
'''

1631


/tmp/ipykernel_6179/681703188.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data[continu_col].mask(cond, inplace=True)


# 计算绝对收益相对市场表现

In [101]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [102]:
product_data = pd.read_csv('product_data.csv')
market_data = pd.read_csv('market_data.csv')

In [103]:
# Utils
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.notebook_repr_html=False  # 表格显示
plt.rcParams['figure.dpi'] = 100  # 图形分辨率
sns.set_theme(style='darkgrid')  # 图形主题
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.rcParams["figure.autolayout"]=True
def plot_value(data):
    plt.figure(figsize=(14,4))
    tmp = pd.DataFrame({sr.name:sr for sr in data}, index=data[0].index)
    sns.lineplot(tmp)
    plt.show()
def date_str_to_month_str(datetime_str):
    return datetime_str[:-3]
def str_return_to_float(str_return):
    return float(str_return[:-1])/100
def time_filter(market_data, idx, mode, period, industry=False):
    origin = pd.to_datetime(market_data.loc[idx]['发行日期'])
    if period == 'month':
        T = pd.Timedelta(value=30, unit='day')
    else:
        T = pd.Timedelta(value=7, unit='day')
    interval = market_data.loc[idx-40:idx+1]
    cond = (origin - pd.to_datetime(interval['发行日期'])) < T
    # industry 是Wind行业名称字符串
    if industry:
        cond  = cond & (interval['Wind行业'] == interval.loc[idx]['Wind行业'])
    if mode == 'mean':
        return interval[cond]['解禁日收益率'].mean()
    else:
        return interval[cond]['解禁日收益率'].sample(frac=3.0, replace=True).mean()
def num_share_in_product(PRODUCT_NAME, product_data):
    share_per_product = []
    for n in PRODUCT_NAME:
        share_per_product.append(product_data[product_data[n].notna()].shape[0])
    print(int(np.mean(share_per_product)))

In [104]:
# 有公司在产品列表中但不再市场数据里，全部为2022年11月末和12月的票
#product_data[~product_data['公司名称'].isin(market_data['名称'])]['公司名称']

## 如果产品内单票在市场中数据缺失，则从同月市场单票中随机选1个作为替代

In [105]:
# 臻享7号和臻享8号因为一大部分成份股来自2022年12月，缺少市场数据，故不做分析
PRODUCT_NAME = ['财富26号', '财富26号权重', '建信整体',
       '建信整体权重', 'FOF1', 'FOF1权重', 'FOF2', 'FOF2权重', 'FOF3', 'FOF3权重',
       '善建FOF1号', '善建FOF1号权重', '善建FOF2号', '善建FOF2号权重', '财富35', '财富35权重',
       '财富38', '财富38权重', '海蓝1号', '海蓝1号权重', '海蓝2号', '海蓝2号权重', '再融资2号',
       '再融资2号权重', '定增精选1期', '定增精选1期权重', '臻享2号', '臻享2号权重', '臻享3号', '臻享3号权重',
       '再融资8号', '再融资8号权重', '多空1号', '多空1号权重', '盛世3期', '盛世3期权重', '再融资1号', '再融资1号权重', '财富42号', '财富42号权重',
       '再融资6号', '再融资6号权重', '盛世4期', '盛世4期权重', '盛世5期', '盛世5期权重', '鸣森1号',
       '鸣森1号权重', '再融资5号', '再融资5号权重', '臻享1号', '臻享1号权重', '臻享6号', '臻享6号权重']
PRODUCT_NAME = sorted([name for name in PRODUCT_NAME if not name.endswith('权重')])

In [106]:
PRODUCT_NAME

['FOF1',
 'FOF2',
 'FOF3',
 '再融资1号',
 '再融资2号',
 '再融资5号',
 '再融资6号',
 '再融资8号',
 '善建FOF1号',
 '善建FOF2号',
 '多空1号',
 '定增精选1期',
 '建信整体',
 '海蓝1号',
 '海蓝2号',
 '盛世3期',
 '盛世4期',
 '盛世5期',
 '臻享1号',
 '臻享2号',
 '臻享3号',
 '臻享6号',
 '财富26号',
 '财富35',
 '财富38',
 '财富42号',
 '鸣森1号']

In [107]:
def market_(product, market_data, product_data, mode='mean', weight=True, period='month'):
    '''
    产品收益相对市场表现
    
    
    params:
    product: string 产品名称
    market_data: DataFrame 市场数据
    product_data: DataFrame 产品数据
    mode: string 取值为'mean‘表示用均值作为收益率，或’mc'表示随机选取单票收益作为收益率
    weight: bool True表示对市场收益率使用产品中对应单票的权重
    period: string 向前取的单票时间范围， 取值为'month'或'week'
    
    return:
    bool 产品在每月收益率相对市场胜率上没有超过50%则返回True
    '''
    share_collection = product_data[product_data[product].notna()].copy()

    share_collection['发行日期'] = share_collection['发行日期'].astype(np.string_)
    share_collection['发行日期'] = share_collection['发行日期'].apply(lambda x:x.decode('UTF-8'))
    share_collection['发行月份'] = share_collection['发行日期'].apply(date_str_to_month_str)
    market_data['发行月份'] = market_data['发行日期'].apply(date_str_to_month_str)
    market_data['解禁日收益率'] = market_data['解禁日浮盈'].apply(str_return_to_float)

    share_collection['解禁日加权收益'] = share_collection['解禁日收益率']*share_collection[product+'权重']
    product_return = share_collection.groupby('发行月份').sum()
    product_value = product_return.cumsum(axis=0)['解禁日加权收益']
    
    share_idx_list = []
    for idx in share_collection.index:
        share = share_collection.loc[idx]
        code = share['代码']
        month = share['发行月份']
        cond1 = (market_data['代码'] == code) & (market_data['发行月份'] == month)
        try:
            share_idx_list.append(market_data[cond1].index[0])
        except:
            share_idx_list.append(market_data[market_data['发行月份'] == '2022-11'].sample(n=1).index[0])

    return_list = []
    # mean模式就算前40个单票中1个月内的票平均收益率，mc模式从前40个票1个月内票有放回的选择全部单票数量并计算平均收益率
    for idx in share_idx_list:
        if idx <= 40:
            return_list.append(market_data.loc[idx]['解禁日收益率'])
        else:
            return_list.append(time_filter(market_data, idx, mode, period))

    market_return = pd.DataFrame({'发行月份':market_data.loc[share_idx_list]['发行月份'], '解禁日收益率':return_list})

    # 考虑产品投资单票权重2
    if weight:
        data = market_return['解禁日收益率'].values*share_collection[product+'权重'].values
        market_return['解禁日加权收益率'] = pd.Series(data, index=market_return.index)
    else:
        market_return['解禁日加权收益率'] = market_return['解禁日收益率']/market_return.shape[0]

    market_rolling_return = market_return.groupby('发行月份').sum()
    market_value = market_rolling_return.cumsum(axis=0)['解禁日加权收益率']
    # 画图
    #data = [product_value, market_value]
    #plot_value(product_value, market_value)

    if market_value.shape != product_value.shape:
        product_value.drop(labels=['2022-12'], inplace=True)
    win_rate = sum(market_value < product_value)/product_value.shape[0]
    if  win_rate < 0.5:
        print('Failure:'+product+'\n')
        print('Product final return rate:'+str(product_value[-1]))
        print('Market final return rate:'+str(market_value[-1]))
        print('Win rate:'+str(win_rate))
        print('=========================================================')
        return True
    else:
        return False

In [108]:
def industry_(product, market_data, product_data, mode='mean', weight=True, period='month'):
    '''
    产品收益相对同行业单票组合表现
    
    
    params:
    product: string 产品名称
    market_data: DataFrame 市场数据
    product_data: DataFrame 产品数据
    mode: string 取值为'mean‘表示用均值作为收益率，或’mc'表示随机选取单票收益作为收益率
    weight: bool True表示对市场收益率使用产品中对应单票的权重
    period: string 向前取的单票时间范围， 取值为'month'或'week'
    
    return:
    bool 产品在每月收益率相对同行业组合胜率上没有超过50%则返回True
    '''
    share_collection = product_data[product_data[product].notna()].copy()

    share_collection['发行日期'] = share_collection['发行日期'].astype(np.string_)
    share_collection['发行日期'] = share_collection['发行日期'].apply(lambda x:x.decode('UTF-8'))
    share_collection['发行月份'] = share_collection['发行日期'].apply(date_str_to_month_str)
    market_data['发行月份'] = market_data['发行日期'].apply(date_str_to_month_str)
    market_data['解禁日收益率'] = market_data['解禁日浮盈'].apply(str_return_to_float)

    share_collection['解禁日加权收益'] = share_collection['解禁日收益率']*share_collection[product+'权重']
    product_return = share_collection[['发行月份','解禁日加权收益']].groupby('发行月份').sum()
    product_value = product_return.cumsum(axis=0)['解禁日加权收益']
    

    share_idx_list = []
    for idx in share_collection.index:
        share = share_collection.loc[idx]
        code = share['代码']
        month = share['发行月份']
        cond1 = (market_data['代码'] == code) & (market_data['发行月份'] == month)
        try:
            share_idx_list.append(market_data[cond1].index[0])
        except:
            share_idx_list.append(market_data[market_data['发行月份'] == '2022-11'].sample(n=1).index[0])

    return_list = []
    # mean模式就算前40个单票中1个月内的票平均收益率，mc模式从前40个票1个月内票有放回的选择全部单票数量并计算平均收益率
    for idx in share_idx_list:
        if idx <= 40:
            return_list.append(market_data.loc[idx]['解禁日收益率'])
        else:
            return_list.append(time_filter(market_data, idx, mode, period, industry=True))

    market_return = pd.DataFrame({'发行月份':market_data.loc[share_idx_list]['发行月份'], \
                                 '解禁日收益率':market_data.loc[share_idx_list]['解禁日收益率']})

    # 考虑产品投资单票权重2
    if weight:
        data = market_return['解禁日收益率'].values*share_collection[product+'权重'].values
        market_return['解禁日加权收益率'] = pd.Series(data, index=market_return.index)
    else:
        market_return['解禁日加权收益率'] = market_return['解禁日收益率']/market_return.shape[0]

    market_rolling_return = market_return.groupby('发行月份').sum()
    market_value = market_rolling_return.cumsum(axis=0)['解禁日加权收益率']
    # 画图
    #data = [product_value, market_value]
    #plot_value(product_value, market_value)

    if market_value.shape != product_value.shape:
        product_value.drop(labels=['2022-12'], inplace=True)
    win_rate = sum(market_value < product_value)/product_value.shape[0]
    if  win_rate < 0.5:
        print('Failure:'+product+'\n')
        print('Product final return rate:'+str(product_value[-1]))
        print('Market final return rate:'+str(market_value[-1]))
        print('Win rate:'+str(win_rate))
        print('=========================================================')
        return True
    else:
        return False

In [109]:
def product_market_strategy(product, market_data, product_data, mode='mean', weight=True, period='month'):
    '''
    产品收益相对市场表现
    
    
    params:
    product: string 产品名称
    market_data: DataFrame 市场数据
    product_data: DataFrame 产品数据
    mode: string 取值为'mean‘表示用均值作为收益率，或’mc'表示随机选取单票收益作为收益率
    weight: bool True表示对市场收益率使用产品中对应单票的权重
    period: string 向前取的单票时间范围， 取值为'month'或'week'
    
    return:
    
    '''
    share_collection = product_data[product_data[product].notna()].copy()

    share_collection['发行日期'] = share_collection['发行日期'].astype(np.string_)
    share_collection['发行日期'] = share_collection['发行日期'].apply(lambda x:x.decode('UTF-8'))
    share_collection['发行月份'] = share_collection['发行日期'].apply(date_str_to_month_str)
    market_data['发行月份'] = market_data['发行日期'].apply(date_str_to_month_str)
    market_data['解禁日收益率'] = market_data['解禁日浮盈'].apply(str_return_to_float)

    share_collection['解禁日加权收益'] = share_collection['解禁日收益率']*share_collection[product+'权重']
    product_return = share_collection.groupby('发行月份').sum()
    product_value = product_return.cumsum(axis=0)['解禁日加权收益']
    
    share_idx_list = []
    for idx in share_collection.index:
        share = share_collection.loc[idx]
        code = share['代码']
        month = share['发行月份']
        cond1 = (market_data['代码'] == code) & (market_data['发行月份'] == month)
        try:
            share_idx_list.append(market_data[cond1].index[0])
        except:
            share_idx_list.append(market_data[market_data['发行月份'] == '2022-11'].sample(n=1).index[0])

    return_list = []
    # mean模式就算前40个单票中1个月内的票平均收益率，mc模式从前40个票1个月内票有放回的选择全部单票数量并计算平均收益率
    for idx in share_idx_list:
        if idx <= 40:
            return_list.append(market_data.loc[idx]['解禁日收益率'])
        else:
            return_list.append(time_filter(market_data, idx, mode, period))

    market_return = pd.DataFrame({'发行月份':market_data.loc[share_idx_list]['发行月份'], \
                                 '解禁日收益率':return_list})

    # 考虑产品投资单票权重
    if weight:
        data = market_return['解禁日收益率'].values*share_collection[product+'权重'].values
        market_return['解禁日加权收益率'] = pd.Series(data, index=market_return.index)
    else:
        market_return['解禁日加权收益率'] = market_return['解禁日收益率']/market_return.shape[0]

    market_rolling_return = market_return.groupby('发行月份').sum()
    market_value = market_rolling_return.cumsum(axis=0)['解禁日加权收益率']
    # 产品实际收益率
    share_collection['实际加权收益'] = share_collection['实际收益/浮盈']*share_collection[product+'权重']
    real_product_return = share_collection.groupby('发行月份').sum()
    real_product_value = real_product_return.cumsum(axis=0)['实际加权收益']
    product_value.name = '产品'
    market_value.name = '市场'
    real_product_value.name = '策略'
    # 画图
    data = [product_value, market_value, real_product_value]
    plot_value(data)

In [110]:
def discount_(product, product_data):
    '''
    产品相对折扣率胜率
    
    
    params:
    product: string 产品名称
    product_data: DataFrame 产品数据
    
    return:
    bool 实际收益在超过50%情况下低于折扣率返回True
    '''
    share_collection = product_data[product_data[product].notna()].copy()
    # data = [share_collection['实际收益/浮盈'], share_collection['折价率']]
    #plot_value(share_collection['实际收益/浮盈'], share_collection['折价率'])
    win = (1 - share_collection['折价率']) * (1 + share_collection['实际收益/浮盈']) > 1
    win_rate = sum(win) / share_collection.shape[0] 
    if win_rate < 0.5:
        return True
    else:
        return False

In [111]:
def strategy_(product, product_data):
    '''
    产品退出策略有效性检验
    
    
    params:
    product: string 产品名称
    product_data: DataFrame 产品数据
    
    return:
    bool 使用退出策略在超过50%情况下提升产品收益则返回True
    '''
    share_collection = product_data[product_data[product].notna()].copy()
    # data = [share_collection['实际收益/浮盈'], share_collection['解禁日收益率']]
    #plot_value(share_collection['实际收益/浮盈'], share_collection['解禁日收益率'])
    length = sum(share_collection['解禁日收益率'] != share_collection['实际收益/浮盈'])
    if length == 0: return False
    win_rate = sum(share_collection['解禁日收益率'] < share_collection['实际收益/浮盈']) / length 
    if win_rate < 0.5:
        print('Product', product)
        print('win_rate:', win_rate)
        print('=========================================================')
        return True
    else:
        return False

In [112]:
def index_(product, product_data, index='沪深300'):
    '''
    产品相对指数胜率
    
    
    params:
    product: string 产品名称
    product_data: DataFrame 产品数据
    index: string 基准指数名称，选项为'沪深300', '中证500', '中证1000', '上证50', '创业板指'
    
    return:
    bool 指数收益在超过50%情况下大于产品绝对收益则返回True
    '''
    share_collection = product_data[product_data[product].notna()].copy()
    #data = [share_collection['实际收益/浮盈'], share_collection[index]]
    #plot_value(share_collection['实际收益/浮盈'], share_collection[index])
    win_rate = sum(share_collection[index] < share_collection['实际收益/浮盈']) / share_collection.shape[0] 
    if win_rate < 0.5:
        print('Product', product)
        print('win_rate:', win_rate)
        print('=========================================================')
        return True
    else:
        return False

In [113]:
def plot_index(PRODUCT_NAME, product_data, weight, index='沪深300'):
    '''
    产品相对指数胜率
    
    
    params:
    PRODUCT_NAME: List(string) 产品名称列表
    product_data: DataFrame 产品数据
    weight: bool 是否有权重
    index: string 指数名称
    
    return:
    产品相对指数收益比
    '''
    plt.figure(figsize=(18,5))
    ratio = []
    for n in PRODUCT_NAME:
        share_collection = product_data[product_data[n].notna()].copy()
        if weight:
            product_return = 1 + (share_collection['实际收益/浮盈']*share_collection[n+'权重']).sum()
            index_return = 1 + (share_collection[index]*share_collection[n+'权重']).sum()
        else:
            product_return = 1 + (share_collection['实际收益/浮盈'] / len(share_collection)).sum()
            index_return = 1 + (share_collection[index] / len(share_collection)).sum()
        ratio.append(product_return/index_return)
    data = sorted([(p, r) for p, r in zip(PRODUCT_NAME, ratio)], key=lambda x:x[1])
    product = [x[0] for x in data]
    ratio = [x[1] for x in data]
    plt.bar(product, ratio)
    plt.plot([1 for i in range(len(ratio))], color='black')
    plt.ylim([0.9, 1.7])
    plt.show()

In [114]:
# index in '沪深300','中证500', '中证1000', '上证50', '创业板指'
#index = '创业板指'
#plot_index(PRODUCT_NAME, product_data, index)

In [115]:
def cause_analyse(PRODUCT_NAME, product_data, market_data, evaluate='model'):
    '''
    产品收益率归因分析
    params:
    PRODUCT_NAME: list of string 产品名列表
    market_data: DataFrame 市场数据
    product_data: DataFrame 产品数据
    evaluate: string 分析因素 ‘market’对比产品与市场表现，’strategy‘对比不同退出策略，‘index’对比产品与指数收益\
    ‘industry’对比产品与同行业单票，‘discount’对比折扣率与产品收益， ‘model'对比LightGBM二分类模型预测与产品
    '''
    if evaluate == 'market':
        failure_list = []
        for product in PRODUCT_NAME:
            failure_list.append(market_(product, market_data, product_data, mode='mean', weight=True, period='month'))
        print('=========================================================')
        print([product for i, product in enumerate(PRODUCT_NAME) if failure_list[i]])
    elif evaluate == 'strategy':
        failure_list = []
        for product in PRODUCT_NAME:
            failure_list.append(strategy_(product, product_data))
        print('=========================================================')
        print([product for i, product in enumerate(PRODUCT_NAME) if failure_list[i]])
    elif evaluate == 'discount':
        failure_list = []
        for product in PRODUCT_NAME:
            failure_list.append(discount_(product, product_data))
        print('=========================================================')
        print([product for i, product in enumerate(PRODUCT_NAME) if failure_list[i]])
    elif evaluate == 'index':
        failure_list = []
        for product in PRODUCT_NAME:
            failure_list.append(index_(product, product_data, index))
        print('=========================================================')
        print([product for i, product in enumerate(PRODUCT_NAME) if failure_list[i]])
    elif evaluate == 'industry':
        failure_list = []
        for product in PRODUCT_NAME:
            failure_list.append(industry_(product, market_data, product_data, mode='mc', weight=False, period='month'))
        print('=========================================================')
        print([product for i, product in enumerate(PRODUCT_NAME) if failure_list[i]])
    else:
        for product in PRODUCT_NAME:
            product_market_strategy(product, market_data, product_data, mode, weight, period)

In [116]:
def plot_winrate(PRODUCT_NAME, market_data, mode='mean', weight=True, period='month'):
    '''
    所有产品解禁日收益率相对市场胜率条形图
    
    
    params:
    PRODUCT_NAME：List(str) 所有产品名称列表
    market_data: DataFrame 市场数据
    '''
    plt.figure(figsize=(20,5))
    ratio = []
    for product in PRODUCT_NAME:
        share_collection = product_data[product_data[product].notna()].copy()

        share_collection['发行日期'] = share_collection['发行日期'].astype(np.string_)
        share_collection['发行日期'] = share_collection['发行日期'].apply(lambda x:x.decode('UTF-8'))
        share_collection['发行月份'] = share_collection['发行日期'].apply(date_str_to_month_str)
        market_data['发行月份'] = market_data['发行日期'].apply(date_str_to_month_str)
        market_data['解禁日收益率'] = market_data['解禁日浮盈'].apply(str_return_to_float)

        share_collection['解禁日加权收益'] = share_collection['解禁日收益率']*share_collection[product+'权重']
        product_value = 1 + share_collection['解禁日加权收益'].sum()

        share_idx_list = []
        for idx in share_collection.index:
            share = share_collection.loc[idx]
            code = share['代码']
            month = share['发行月份']
            cond1 = (market_data['代码'] == code) & (market_data['发行月份'] == month)
            try:
                share_idx_list.append(market_data[cond1].index[0])
            except:
                share_idx_list.append(market_data[market_data['发行月份'] == '2022-11'].sample(n=1).index[0])

        return_list = []
        # mean模式就算前40个单票中1个月内的票平均收益率，mc模式从前40个票1个月内票有放回的选择全部单票数量并计算平均收益率
        for idx in share_idx_list:
            if idx <= 40:
                return_list.append(market_data.loc[idx]['解禁日收益率'])
            else:
                return_list.append(time_filter(market_data, idx, mode, period))

        market_return = pd.DataFrame({'发行月份':market_data.loc[share_idx_list]['发行月份'], \
                                     '解禁日收益率':return_list})

        # 考虑产品投资单票权重2
        if weight:
            data = market_return['解禁日收益率'].values*share_collection[product+'权重'].values
            market_return['解禁日加权收益率'] = pd.Series(data, index=market_return.index)
        else:
            market_return['解禁日加权收益率'] = market_return['解禁日收益率']/market_return.shape[0]

        market_value = 1 + market_return['解禁日加权收益率'].sum()
        ratio.append(product_value / market_value)
    data = sorted([(p, r) for p, r in zip(PRODUCT_NAME, ratio)], key=lambda x:x[1])
    product = [x[0] for x in data]
    ratio = [x[1] for x in data]
    plt.bar(product, ratio)
    plt.plot([1 for i in range(len(ratio))], color='black')
    plt.ylim([0.8, 1.2])
    plt.show()

In [117]:
mode = 'mean'
weight= True
period = 'month'

In [118]:
#plot_winrate(PRODUCT_NAME, market_data, mode, weight, period)

In [119]:
# 策略无效：['FOF3', '再融资2号', '再融资5号', '再融资6号', '定增精选1期', '海蓝1号', '盛世4期', '盛世5期']
# 绝对收益低于折扣率：几乎所有
cause_analyse(PRODUCT_NAME, product_data, market_data, evaluate='market')

Failure:再融资5号

Product final return rate:0.03810910278710132
Market final return rate:0.02785912138833151
Win rate:0.3333333333333333
Failure:再融资8号

Product final return rate:0.07298686309109605
Market final return rate:0.0826060286077426
Win rate:0.0
Failure:善建FOF1号

Product final return rate:0.06150873316477015
Market final return rate:0.07017528161576983
Win rate:0.4
Failure:多空1号

Product final return rate:0.010830468745799444
Market final return rate:0.040132330186812594
Win rate:0.0
Failure:定增精选1期

Product final return rate:-0.0008754899572693611
Market final return rate:0.023210301576782236
Win rate:0.0
Failure:建信整体

Product final return rate:0.11102776789877465
Market final return rate:0.32361044690530044
Win rate:0.045454545454545456
Failure:海蓝1号

Product final return rate:0.17233145811703668
Market final return rate:0.30106250851110905
Win rate:0.3333333333333333
Failure:海蓝2号

Product final return rate:-0.0645972040646026
Market final return rate:0.11878672036505479
Win rate:0

## 相对市场表现
1+1['再融资6号', '善建FOF1号', '善建FOF2号', '定增精选1期', '建信整体', '盛世3期', '臻享2号']

1+0 ['再融资5号', '再融资8号', '善建FOF1号', '善建FOF2号', '多空1号', '定增精选1期', '海蓝1号', '海蓝2号', '盛世4期', '臻享2号', '臻享3号', '臻享6号', '财富26号']

0+1 ['再融资6号', '善建FOF1号', '定增精选1期', '建信整体', '盛世3期', '盛世5期', '臻享2号']

0+0 ['再融资5号', '再融资8号', '善建FOF1号', '善建FOF2号', '多空1号', '定增精选1期', '海蓝1号', '海蓝2号', '盛世4期', '臻享2号', '臻享3号', '臻享6号', '财富26号']

## 相对行业表现
1+1['再融资6号', '善建FOF1号', '善建FOF2号', '定增精选1期', '建信整体', '盛世3期', '盛世5期', '臻享1号', '臻享2号']

1+0 ['再融资5号', '再融资8号', '善建FOF1号', '善建FOF2号', '多空1号', '定增精选1期', '海蓝1号', '海蓝2号', '盛世4期', '臻享2号', '臻享3号', '臻享6号', '财富26号']

0+1 ['再融资6号', '善建FOF1号', '定增精选1期', '建信整体', '盛世3期', '盛世5期', '臻享1号', '臻享2号']

0+0 ['再融资5号', '再融资8号', '善建FOF1号', '善建FOF2号', '多空1号', '定增精选1期', '海蓝1号', '海蓝2号', '盛世4期', '臻享2号', '臻享3号', '臻享6号', '财富26号']